In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display
from datetime import datetime
import os
import random
from openpyxl import load_workbook
import re

In [2]:
file_path = 'B2B SaaS Companies and Contacts.xlsx' 
data = pd.read_excel(file_path)

In [3]:

display(data.head())

,Company Name,"City, State",Industry,Employees on Linkedin,Company Size,Anual Revenue,Website,Domain,Linkedin Link,Company ID
0,FloWorks,"Pasadena, Texas",Automation Machinery Manufacturing,123.0,"501-1,000 employees",$269.7M,www.gofloworks.com,gofloworks.com,https://www.linkedin.com/company/floworks-inte...,6623089.0
1,Riviana Foods,NaN,NaN,NaN,NaN,NaN,NaN,riviana.com,NaN,NaN
2,Nexeo Plastics,NaN,NaN,NaN,NaN,NaN,NaN,nexeoplastics.com,NaN,NaN
3,Kuraray America,NaN,NaN,NaN,NaN,NaN,NaN,kuraray.us.com,NaN,NaN
4,Puffer-Sweiven,NaN,NaN,NaN,NaN,NaN,NaN,puffer.com,NaN,NaN


In [ ]:
def get_linkedin_data(company_name):
    formatted_name = company_name.replace(' ', '-').lower()
    search_url = f"https://www.linkedin.com/company/{formatted_name}"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(search_url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            #Extract city,state
            city_state_elem = soup.find("div", {'data-test-id':'about-us__headquarters'}).find('dd') if soup.find('div', {'data-test-id': 'about-us__headquarters'}) else None
            city_state = city_state_elem.text.strip() if city_state_elem else 'N/A'
            
            #Extract Address
            address_elem = soup.find("div", id="address-0")
            if address_elem:
                paragraphs = address_elem.find_all('p')
                if len(paragraphs) > 1:
                    address = paragraphs[1].text.strip() 

            # Extract industry
            industry_elem = soup.find('h2', class_='top-card-layout__headline break-words font-sans text-md leading-open text-color-text')
            industry = industry_elem.text.strip() if industry_elem else 'N/A'

            # Extract employees count
            company_size_elem = soup.find('div', {'data-test-id': 'about-us__size'}).find('dd') if soup.find('div', {'data-test-id': 'about-us__size'}) else None
            company_size = company_size_elem.text.strip() if company_size_elem else 'N/A'
         
            # Extract company size
            employees_on_linkedin_elem = soup.find('p', class_="face-pile__text self-center font-sans text-sm link-styled hover:underline")
            employees_on_linkedin = employees_on_linkedin_elem.text.strip() if employees_on_linkedin_elem else 'N/A'
            
            # Extract annual revenue
            annual_revenue = 'N/A' 

            # Extract Website company
            website_elem = soup.find('a', class_='link-no-visited-state hover:no-underline')
            website = website_elem.text.strip() if website_elem else ''

            # Extract Domain
            domain = website.replace('https://', '').replace('http://', '').replace('www.', '') if website else 'N/A'
            
            # Extract LinkedIn link
            linkedin = search_url
            
            # Extract ID
            company_id = random.randint(1000000, 9999999)

            return city_state, address, industry, employees_on_linkedin, company_size, annual_revenue, website, domain, linkedin, company_id
        return 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', search_url, 'N/A'
    except Exception as e:
        print(f"Error fetching data for {company_name}: {e}")
        return 'Not found', 'Not found', 'Not found', 'Not found', 'Not found', 'Not found', 'Not found', 'Not found', search_url, 'Not found'

data[['City, State', 'Address', 'Industry', 'Employees on Linkedin', 'Company Size', 'Anual Revenue', 'Website', 'Domain', 'Linkedin Link', 'Company ID']] = data['Company Name'].apply(lambda x: pd.Series(get_linkedin_data(x)))

def save_to_excel(data, filename='enriched_company_data.xlsx'):
    if os.path.exists(filename):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        new_filename = f"{filename.rsplit('.', 1)[0]}_{timestamp}.xlsx"
        data.to_excel(new_filename, index=False)
        print(f"File already exists. Saved as {new_filename}")
    data.to_excel(filename, index=False)
    print(f"File saved as {filename}")
    new_filename = filename

    workbook = load_workbook(new_filename)
    worksheet = workbook.active

    for column in worksheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        adjusted_width = max_length + 2
        worksheet.column_dimensions[column_letter].width = adjusted_width

    workbook.save(new_filename)
    print(f"Adjusted column widths and saved file as {new_filename}")

save_to_excel(data)

Error fetching data for Riviana Foods: cannot access local variable 'address' where it is not associated with a value
Error fetching data for Silicon Labs: cannot access local variable 'address' where it is not associated with a value
Error fetching data for AZZ: cannot access local variable 'address' where it is not associated with a value
Error fetching data for Kraton: cannot access local variable 'address' where it is not associated with a value
Error fetching data for CECO: cannot access local variable 'address' where it is not associated with a value
Error fetching data for NI: cannot access local variable 'address' where it is not associated with a value
Error fetching data for Lexus: cannot access local variable 'address' where it is not associated with a value
Error fetching data for ChampionX: cannot access local variable 'address' where it is not associated with a value
Error fetching data for Arcosa: cannot access local variable 'address' where it is not associated with a v

In [5]:
for index, row in data.iterrows():
    company_info = ", ".join([f"{col}: {row[col]}" for col in data.columns])
    print(f"{index}: {company_info}")

0: Company Name: FloWorks, City, State: Zuid-Scharwoude, North Holland, Industry: Business Consulting and Services, Employees on Linkedin: View 1 employee, Company Size: 1 employee, Anual Revenue: $269.7M, Website: http://www.floworks.eu, Domain: floworks.eu, Linkedin Link: https://www.linkedin.com/company/floworks, Company ID: 3505082, Address: Zuid-Scharwoude, North Holland 1722, NL, Annual Revenue: N/A
1: Company Name: Riviana Foods, City, State: Not found, Industry: Not found, Employees on Linkedin: Not found, Company Size: Not found, Anual Revenue: nan, Website: Not found, Domain: Not found, Linkedin Link: https://www.linkedin.com/company/riviana-foods, Company ID: Not found, Address: Not found, Annual Revenue: Not found
2: Company Name: Nexeo Plastics, City, State: N/A, Industry: N/A, Employees on Linkedin: N/A, Company Size: N/A, Anual Revenue: nan, Website: N/A, Domain: N/A, Linkedin Link: https://www.linkedin.com/company/nexeo-plastics, Company ID: N/A, Address: N/A, Annual Re

In [6]:
display(data)

,Company Name,"City, State",Industry,Employees on Linkedin,Company Size,Anual Revenue,Website,Domain,Linkedin Link,Company ID,Address,Annual Revenue
0,FloWorks,"Zuid-Scharwoude, North Holland",Business Consulting and Services,View 1 employee,1 employee,$269.7M,http://www.floworks.eu,floworks.eu,https://www.linkedin.com/company/floworks,3505082,"Zuid-Scharwoude, North Holland 1722, NL",N/A
1,Riviana Foods,Not found,Not found,Not found,Not found,NaN,Not found,Not found,https://www.linkedin.com/company/riviana-foods,Not found,Not found,Not found
2,Nexeo Plastics,N/A,N/A,N/A,N/A,NaN,N/A,N/A,https://www.linkedin.com/company/nexeo-plastics,N/A,N/A,N/A
3,Kuraray America,N/A,N/A,N/A,N/A,NaN,N/A,N/A,https://www.linkedin.com/company/kuraray-america,N/A,N/A,N/A
4,Puffer-Sweiven,"Stafford, Texas",Automation Machinery Manufacturing,Access all 659 employees,"501-1,000 employees",NaN,https://www.puffer.com,puffer.com,https://www.linkedin.com/company/puffer-sweiven,7009482,"Stafford, Texas 77477, US",N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
68,Oldcastle BuildingEnvelope,"Dallas, TX",Wholesale Building Materials,"Discover all 1,836 employees","5,001-10,000 employees",NaN,http://www.obe.com/,obe.com/,https://www.linkedin.com/company/oldcastle-bui...,1049942,Suite 1050,N/A
69,Trinity Industries,"Dallas, Texas",Manufacturing,"Discover all 3,694 employees","10,001+ employees",NaN,http://jobs.trin.net,jobs.trin.net,https://www.linkedin.com/company/trinity-indus...,9171619,"Dallas, Texas 75254, US",N/A
70,ATI,N/A,N/A,N/A,N/A,NaN,N/A,N/A,https://www.linkedin.com/company/ati,N/A,N/A,N/A
71,Matheson Tri-Gas,N/A,N/A,N/A,N/A,NaN,N/A,N/A,https://www.linkedin.com/company/matheson-tri-gas,N/A,N/A,N/A
